In [ ]:
# Current directory
import os
os.chdir('F:\Work\Experiment\pLM4ACE\model')

### 导入数据

In [ ]:
import numpy as np
import pandas as pd

X_new = pd.read_csv(r"fusion_features\Data\features_select\PCA_All.csv", index_col=False, header=None)
y_new = pd.read_csv("fusion_features\Data\label.csv", index_col=False, header=None)

print(X_new.shape)
print(y_new.shape)
print(np.count_nonzero(y_new==0))
print(np.count_nonzero(y_new==1))

X_new = np.array(X_new, 'float32')
y_new = np.array(y_new)

### 测试

In [ ]:
# 10折交叉验证
import os,sys,math
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import scale
from keras import backend as K
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Lambda, Concatenate, Multiply
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import adam_v2 #Adam 改为 adam_v2
from sklearn.metrics import confusion_matrix
import math
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split
X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)


# train_set = scale(X_train_whole)
[sample_num, input_dimwx]=np.shape(X_train_whole)
X = np.reshape(X_train_whole, (sample_num, 1, input_dimwx, 1))
y = y_train_whole


def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

def Capsule_Layer():
    img = Input(shape=(1,input_dimwx,1))
    x = Conv2D(filters=64, kernel_size=(1,9), strides=2, padding='valid', name='conv1')(img)
    x = LeakyReLU()(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    x = Conv2D(filters=32, kernel_size=(1,9), strides=2, padding='valid', name='conv1')(img)
    x = LeakyReLU()(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    """
    NOTE: Capsule architecture starts from here.
    """
    ##### primarycaps coming first ##### 
    x = Conv2D(filters=32, kernel_size=(1,3), strides=2, padding='valid', name='primarycap_conv2')(x)    
    [aa,bb,cc,dd] = x.shape
    numx = int(cc)
    x = Reshape(target_shape=[-1, numx], name='primarycap_reshape')(x)
    x = Lambda(squash, name='primarycap_squash')(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    ##### digitcaps are here ##### 
    x = Flatten()(x)
    uhat = Dense(32, kernel_initializer='he_normal', bias_initializer='zeros', name='uhat_digitcaps')(x)
    c = Activation('softmax', name='softmax_digitcaps1')(uhat) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    """
    NOTE: Squashing the capsule outputs creates severe blurry artifacts, thus we replace it with Leaky ReLu.
    """
    s_j = LeakyReLU()(x)
    ##### we will repeat the routing part 2 more times (num_routing=3) to unfold the loop
    c = Activation('softmax', name='softmax_digitcaps2')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)

    c = Activation('softmax', name='softmax_digitcaps3')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)
    
    c = Activation('softmax', name='softmax_digitcaps4')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)

    pred = Dense(2, activation='sigmoid')(s_j)
    model = Model (img, pred)
    # model.summary()
    model.compile(loss='binary_crossentropy', optimizer=adam_v2.Adam (0.0002, 0.5), metrics=['binary_accuracy'])
    return model


BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]

mean_recall = np.linspace(0, 1, 100)
all_precision = []

base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
# 新的TPR集合
interp_tpr_collection = []


def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)


def to_categorical(y, nb_classes=None):
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1
    return Y


skf = StratifiedKFold(n_splits=10)
for train, test in skf.split(X, y):
    X_train, X_valid, y_train, y_valid = np.take(X, train.tolist(), axis=0), np.take(X, test.tolist(), axis=0), np.take(y, train.tolist(), axis=0), np.take(y, test.tolist(), axis=0)
    y_train = to_categorical(y_train)
    cv_clf = Capsule_Layer()
    hist = cv_clf.fit(X_train, y_train, batch_size=64, epochs=60)
    y_score = cv_clf.predict(X_valid)
    y_class = categorical_probas_to_classes(y_score)
    TP, FP, FN, TN = confusion_matrix(y_valid, y_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    # ROC curve
    fpr, tpr, _ = roc_curve(y_valid, y_score[:, 1])
    interp_tpr = np.interp(base_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tpr_collection.append(interp_tpr)
    auc_roc = auc(fpr, tpr)
    AUC_collecton.append(auc_roc)
    # PR curve
    precision, recall, _ = precision_recall_curve(y_valid, y_score[:, 1])
    average_precision = average_precision_score(y_valid, y_score[:, 1])
    recall = np.flipud(recall)
    precision = np.flipud(precision)

    mean_precision = np.interp(mean_recall, recall, precision)
    all_precision.append(mean_precision)
    AP.append(average_precision)

# 输出结果
print(round(statistics.mean(BACC_collecton),3),'±',round(statistics.stdev(BACC_collecton),3))
print(round(statistics.mean(Sn_collecton),3),'±',round(statistics.stdev(Sn_collecton),3))
print(round(statistics.mean(Sp_collecton),3),'±',round(statistics.stdev(Sp_collecton),3))
print(round(statistics.mean(MCC_collecton),3),'±',round(statistics.stdev(MCC_collecton),3))
print(round(statistics.mean(AUC_collecton),3),'±',round(statistics.stdev(AUC_collecton),3))
print(round(statistics.mean(AP),3),'±',round(statistics.stdev(AP),3))

# 在所有交叉验证循环结束后，计算TPR的均值
mean_tpr = np.mean(interp_tpr_collection, axis=0)
mean_tpr[-1] = 1.0

# Calculate the mean precision
mean_precision = np.mean(all_precision, axis=0)

# 保存ROC曲线相关参数
np.savez(r'Draw graphics\ROC curve\PCA_All\Capsule_cross_vaild.npz', fpr=base_fpr, tpr=mean_tpr, roc_auc=AUC_collecton)

# 保存PR曲线相关参数
np.savez(r'Draw graphics\PR curve\PCA_All\Capsule_cross_vaild.npz', recall=mean_recall, precision=mean_precision, average_precision=AP)

# 绘制ROC曲线
plt.figure()
lw = 2
plt.plot(base_fpr, mean_tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % np.mean(AUC_collecton))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('10 k-fold cross vaild')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# 独立测试集
import os,sys,math
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import scale
from keras import backend as K
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Lambda, Concatenate, Multiply
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import adam_v2 #Adam 改为 adam_v2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math


# X_new = scale(X_new)
[sample_num, input_dimwx]=np.shape(X_new)
X = np.reshape(X_new, (sample_num, 1, input_dimwx, 1))
y = y_new


def squash(vectors, axis=-1):
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

def Capsule_Layer():
    img = Input(shape=(1,input_dimwx,1))
    x = Conv2D(filters=64, kernel_size=(1,9), strides=2, padding='valid', name='conv1')(img)
    x = LeakyReLU()(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    x = Conv2D(filters=32, kernel_size=(1,9), strides=2, padding='valid', name='conv1')(img)
    x = LeakyReLU()(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    """
    NOTE: Capsule architecture starts from here.
    """
    ##### primarycaps coming first ##### 
    x = Conv2D(filters=32, kernel_size=(1,3), strides=2, padding='valid', name='primarycap_conv2')(x)    
    [aa,bb,cc,dd] = x.shape
    numx = int(cc)
    x = Reshape(target_shape=[-1, numx], name='primarycap_reshape')(x)
    x = Lambda(squash, name='primarycap_squash')(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    ##### digitcaps are here ##### 
    x = Flatten()(x)
    uhat = Dense(32, kernel_initializer='he_normal', bias_initializer='zeros', name='uhat_digitcaps')(x)
    c = Activation('softmax', name='softmax_digitcaps1')(uhat) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    """
    NOTE: Squashing the capsule outputs creates severe blurry artifacts, thus we replace it with Leaky ReLu.
    """
    s_j = LeakyReLU()(x)
    ##### we will repeat the routing part 2 more times (num_routing=3) to unfold the loop
    c = Activation('softmax', name='softmax_digitcaps2')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)

    c = Activation('softmax', name='softmax_digitcaps3')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)
    
    c = Activation('softmax', name='softmax_digitcaps4')(s_j) # softmax will make sure that each weight c_ij is a non-negative number and their sum equals to one
    c = Dense(32)(c) # compute s_j
    x = Multiply()([uhat, c])
    s_j = LeakyReLU()(x)

    pred = Dense(2, activation='sigmoid')(s_j)
    model = Model (img, pred)
    # model.summary()
    model.compile(loss='binary_crossentropy', optimizer=adam_v2.Adam (0.0002, 0.5), metrics=['binary_accuracy'])
    return model


BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]
mean_recall = np.linspace(0, 1, 100)
all_precision = []

base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
# 新的TPR集合
interp_tpr_collection = []


def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)


def to_categorical(y, nb_classes=None):
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1
    return Y


for i in range(10):
    # dataset splitting
    X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X, y, test_size=0.2, random_state=i)
    y_train_whole = to_categorical(y_train_whole)
    cv_clf = Capsule_Layer()
    hist = cv_clf.fit(X_train_whole, y_train_whole, batch_size=64, epochs=60)
    y_score = cv_clf.predict(X_ind_test)
    y_class = categorical_probas_to_classes(y_score)
    TP, FP, FN, TN = confusion_matrix(y_ind_test, y_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    # ROC curve
    fpr, tpr, _ = roc_curve(y_ind_test, y_score[:, 1])
    interp_tpr = np.interp(base_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    interp_tpr_collection.append(interp_tpr)
    auc_roc = auc(fpr, tpr)
    AUC_collecton.append(auc_roc)
    # PR curve
    precision, recall, _ = precision_recall_curve(y_ind_test, y_score[:, 1])
    average_precision = average_precision_score(y_ind_test, y_score[:, 1])
    recall = np.flipud(recall)
    precision = np.flipud(precision)

    mean_precision = np.interp(mean_recall, recall, precision)
    all_precision.append(mean_precision)
    AP.append(average_precision)

# 输出结果
print(round(statistics.mean(BACC_collecton),3),'±',round(statistics.stdev(BACC_collecton),3))
print(round(statistics.mean(Sn_collecton),3),'±',round(statistics.stdev(Sn_collecton),3))
print(round(statistics.mean(Sp_collecton),3),'±',round(statistics.stdev(Sp_collecton),3))
print(round(statistics.mean(MCC_collecton),3),'±',round(statistics.stdev(MCC_collecton),3))
print(round(statistics.mean(AUC_collecton),3),'±',round(statistics.stdev(AUC_collecton),3))
print(round(statistics.mean(AP),3),'±',round(statistics.stdev(AP),3))

# 在所有交叉验证循环结束后，计算TPR的均值
mean_tpr = np.mean(interp_tpr_collection, axis=0)
mean_tpr[-1] = 1.0

# Calculate the mean precision
mean_precision = np.mean(all_precision, axis=0)

# 保存ROC曲线相关参数
# np.savez(r'Draw graphics\ROC curve\PCA_All\Capsule_Indenpendence.npz', fpr=base_fpr, tpr=mean_tpr, roc_auc=AUC_collecton)

# 保存PR曲线相关参数
# np.savez(r'Draw graphics\PR curve\PCA_All\Capsule_Indenpendence.npz', recall=mean_recall, precision=mean_precision, average_precision=AP)

# # 绘制ROC曲线
# plt.figure()
# lw = 2
# plt.plot(base_fpr, mean_tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % np.mean(AUC_collecton))
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([-0.05, 1.05])
# plt.ylim([-0.05, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('10 k-fold cross vaild')
# plt.legend(loc="lower right")
# plt.show()

# 绘制PR曲线
plt.figure(facecolor='white')
lw=1

# 设置字体
font = {'family': 'Times New Roman', 'size': 12}
plt.rc('font',family='Times New Roman')

# 绘制曲线
plt.step(mean_recall, mean_precision, color='#F97059', alpha=1, where='post', lw=lw, label='LR (AUPR=%0.4f)' % np.mean(AP))

# 设置图表属性
plt.xlabel('Recall', fontsize=14, weight='bold')
plt.ylabel('Precision', fontsize=14, weight='bold')
plt.title('Independence test - PR', fontsize=18, weight='bold')
plt.legend(loc="lower left", fontsize=10)
plt.show()


### 结尾